# RNNによる乗客数の予測

In [ ]:
from passengers import Passengers, Line
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
file_name = 'data/t091307.xlsx'
line = '田園都市線'
skip_rows = list(range(0, 11)) + list(range(12, 15))
use_cols = list(range(1, 100))
denen = Line(file_name, line, skip_rows, use_cols)

ps = Passengers(line=denen)
display(ps.passengers.head())

# RNN

In [ ]:
df = ps.passengers['総数']

span = 12
data = []
target = []

for i in range(len(df) - span):
    data.append(df[i : i + span])
    target.append(df[i + span])
    
X = np.array(data).reshape(len(data), span, 1)
Y = np.array(target).reshape(len(data), 1)
X = X/10000000
Y = Y/10000000

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.1)

In [ ]:
n_in = len(X[0][0]) 
n_hidden = 10
n_out = len(Y[0])

def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model = Sequential()
model.add(GRU(n_hidden,
              input_shape=(span, n_in)
             ))
model.add(Dense(n_out))
model.add(Activation('linear'))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [ ]:
epochs = 500
batch_size = 6

hist = model.fit(X_train, Y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_validation, Y_validation),
                 callbacks=[early_stopping])

In [ ]:
def plot_history(history):
    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()

plot_history(hist)  

In [ ]:
def forecast(model, x, n, span):
    model.reset_states()
    Y = []
    x_tmp = x
    for i in range(n):
        x_tmp = x_tmp.reshape(1, span, 1)
        y = model.predict(x_tmp)
        Y.append(y)

        x_tmp = x_tmp.reshape(span, 1)
        x_tmp = np.vstack((x_tmp, y))
        x_tmp = np.delete(x_tmp, 0, 0)
        
    model.reset_states()
    return Y

In [ ]:
Y_pred = forecast(model, X_test[0], len(X_test), span)

In [ ]:
Y_test_pred = model.predict(X_test)

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(Y_test, label='original')
plt.plot(np.array([Y_pred]).reshape(len(Y_pred),1), label='forecast')
plt.plot(Y_test_pred, label='predict')
plt.legend(loc='best')
plt.show()

* うまくいっていないが時間がないので、後で確認